In [3]:
from theano.sandbox import cuda
cuda.use('gpu0')

import matplotlib.pyplot as plt
%matplotlib inline

#utils modules contains lots of helper functions from the fast.ai course
import utils; reload(utils)
from utils import *

# Forcing us to use print function and proper division sign
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [4]:
path = '../kaggle/state-farm-distracted-driver-detection'
sample_dir = path + '/sample'

# Load images

In [5]:
from PIL import Image
import numpy as np # linear algebra
img_size_1D = 224
img_target_size = (img_size_1D, img_size_1D)
img_batch_size = 64

def load_in_batches(dir_name, gen=image.ImageDataGenerator(), shuffle=False, 
                    batch_size=4, class_mode='categorical', target_size=img_target_size):
    return gen.flow_from_directory(dir_name, target_size=target_size, shuffle=shuffle, batch_size=batch_size)

In [6]:
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, batch_size=img_batch_size)
sample_val_batches = load_in_batches(sample_dir + '/validation', shuffle=False, batch_size=img_batch_size * 2)

Found 1803 images belonging to 10 classes.
Found 448 images belonging to 10 classes.


# Using a simple model

In [7]:
# Let's import everything we will need from Keras
from keras.models import Sequential
from keras.layers import Lambda, BatchNormalization, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [13]:
num_classes = sample_train_batches.nb_class

In [14]:
def linear_model():
    model = Sequential()
    model.add(BatchNormalization(axis=1, input_shape=(3, img_size_1D, img_size_1D)))
   
    model.add(Flatten())       
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model    

In [15]:
model = linear_model()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_2 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 150528)        0           batchnormalization_2[0][0]       
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            1505290     flatten_2[0][0]                  
Total params: 1505296
____________________________________________________________________________________________________


In [16]:
# Start with a low enough learning rate
model.optimizer.lr = 10e-5
# Now train it for a few epochs
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=3, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/3
1803/1803 [==============================] - 33s - loss: 4.9884 - acc: 0.3272 - val_loss: 8.9146 - val_acc: 0.1875
Epoch 2/3
1803/1803 [==============================] - 22s - loss: 2.9969 - acc: 0.6578 - val_loss: 6.7202 - val_acc: 0.2612
Epoch 3/3
1803/1803 [==============================] - 23s - loss: 2.5178 - acc: 0.7759 - val_loss: 4.9194 - val_acc: 0.3661


In [17]:
# Increase the learning rate as we seem to be on the right track
model.optimizer.lr = 10e-3
# Now train it for a few more epochs
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=4, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/4
1803/1803 [==============================] - 29s - loss: 2.4346 - acc: 0.8209 - val_loss: 4.9203 - val_acc: 0.3482
Epoch 2/4
1803/1803 [==============================] - 23s - loss: 2.1354 - acc: 0.8630 - val_loss: 5.1341 - val_acc: 0.3080
Epoch 3/4
1803/1803 [==============================] - 24s - loss: 2.3261 - acc: 0.8575 - val_loss: 4.7532 - val_acc: 0.3705
Epoch 4/4
1803/1803 [==============================] - 24s - loss: 2.1903 - acc: 0.8641 - val_loss: 4.5360 - val_acc: 0.3705


At this stage, our model's accuracy on the validation set seems to be stagnating - and a bit unstable. The training set accuracy is going down quite well on its side, showing signs of overfit. 
We should look at a more powerful model to see if we can decrease the overfit and increase its generic application.

# Using a simple ConvNet
The convnet contains only two convolutional layers, and is therefore not very deep.

In [18]:
def simple_convnet():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,img_size_1D,img_size_1D)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Flatten(),
        # We could try changing this - not sure it would massively change the results though    
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [19]:
model = simple_convnet()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_3 (BatchNormal(None, 3, 224, 224)   6           batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 222, 222)  896         batchnormalization_3[0][0]       
____________________________________________________________________________________________________
batchnormalization_4 (BatchNormal(None, 32, 222, 222)  64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 74, 74)    0           batchnormalization_4[0][0]       
___________________________________________________________________________________________

In [20]:
model = simple_convnet()

model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=3, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/3
1803/1803 [==============================] - 33s - loss: 2.1803 - acc: 0.3095 - val_loss: 2.4526 - val_acc: 0.1027
Epoch 2/3
1803/1803 [==============================] - 28s - loss: 1.1760 - acc: 0.6522 - val_loss: 2.2902 - val_acc: 0.1317
Epoch 3/3
1803/1803 [==============================] - 25s - loss: 0.6809 - acc: 0.8619 - val_loss: 2.3412 - val_acc: 0.1406


In [21]:
model.optimizer.lr = 10e-3

model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=4, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/4
1803/1803 [==============================] - 32s - loss: 0.4437 - acc: 0.9357 - val_loss: 2.3879 - val_acc: 0.1719
Epoch 2/4
1803/1803 [==============================] - 26s - loss: 0.3073 - acc: 0.9656 - val_loss: 2.3894 - val_acc: 0.2009
Epoch 3/4
1803/1803 [==============================] - 28s - loss: 0.2305 - acc: 0.9778 - val_loss: 2.3776 - val_acc: 0.2031
Epoch 4/4
1803/1803 [==============================] - 27s - loss: 0.1670 - acc: 0.9922 - val_loss: 2.3154 - val_acc: 0.2143


No luck at all here. Our model overfits even more and converges much faster towards perfect accuracy on training sample. We need to make it work harder to spend more time distilling the import features that will enable it to improve accuracy on the validation sample set.
We will use data augmentation for that.

## Data augmentation
We need to try different types of data augmentation, one at a time, to determine what parameters work

In [22]:
# But first, let's combine the steps above into a function
def create_and_run_simple_convnet(t_batch=sample_train_batches, v_batch=sample_val_batches):
    model = simple_convnet()

    model.fit_generator(t_batch, samples_per_epoch=t_batch.nb_sample, nb_epoch=5, 
                       validation_data=v_batch, nb_val_samples=v_batch.nb_sample, verbose=1)

    model.optimizer.lr = 10e-3

    model.fit_generator(t_batch, samples_per_epoch=t_batch.nb_sample, nb_epoch=2, 
                       validation_data=v_batch, nb_val_samples=v_batch.nb_sample, verbose=1)
    
    return model

#### Width_Shift_Augmentations
Here is image is moved left and right

In [23]:
gen_ws = image.ImageDataGenerator(width_shift_range=0.15)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_ws, batch_size=img_batch_size)

Found 1803 images belonging to 10 classes.


In [24]:
create_and_run_simple_convnet()

Epoch 1/5
1803/1803 [==============================] - 32s - loss: 2.2328 - acc: 0.2806 - val_loss: 2.3781 - val_acc: 0.1406
Epoch 2/5
1803/1803 [==============================] - 25s - loss: 1.1289 - acc: 0.6800 - val_loss: 2.4277 - val_acc: 0.1607
Epoch 3/5
1803/1803 [==============================] - 23s - loss: 0.6476 - acc: 0.8874 - val_loss: 2.5304 - val_acc: 0.1473
Epoch 4/5
1803/1803 [==============================] - 27s - loss: 0.4145 - acc: 0.9440 - val_loss: 2.5956 - val_acc: 0.1518
Epoch 5/5
1803/1803 [==============================] - 25s - loss: 0.2855 - acc: 0.9756 - val_loss: 2.6248 - val_acc: 0.1585
Epoch 1/2
1803/1803 [==============================] - 30s - loss: 0.1906 - acc: 0.9900 - val_loss: 2.6090 - val_acc: 0.1607
Epoch 2/2
1803/1803 [==============================] - 27s - loss: 0.1469 - acc: 0.9945 - val_loss: 2.5739 - val_acc: 0.1763


#### Height Shift Augmentations
Here is image is moved up and down

In [25]:
gen_hs = image.ImageDataGenerator(height_shift_range=0.05)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_hs, batch_size=img_batch_size)
create_and_run_simple_convnet()

Found 1803 images belonging to 10 classes.
Epoch 1/5
1803/1803 [==============================] - 33s - loss: 2.2395 - acc: 0.2845 - val_loss: 2.4512 - val_acc: 0.1272
Epoch 2/5
1803/1803 [==============================] - 27s - loss: 1.1973 - acc: 0.6556 - val_loss: 2.2927 - val_acc: 0.1496
Epoch 3/5
1803/1803 [==============================] - 26s - loss: 0.7129 - acc: 0.8647 - val_loss: 2.3315 - val_acc: 0.1161
Epoch 4/5
1803/1803 [==============================] - 26s - loss: 0.4591 - acc: 0.9379 - val_loss: 2.3608 - val_acc: 0.1116
Epoch 5/5
1803/1803 [==============================] - 26s - loss: 0.3085 - acc: 0.9717 - val_loss: 2.3672 - val_acc: 0.1138
Epoch 1/2
1803/1803 [==============================] - 31s - loss: 0.2163 - acc: 0.9878 - val_loss: 2.3345 - val_acc: 0.1295
Epoch 2/2
1803/1803 [==============================] - 27s - loss: 0.1628 - acc: 0.9956 - val_loss: 2.2856 - val_acc: 0.1696


#### Shear Angles Augmentations
In a shear transformation, points are proportionally displaced to map onto a new plane whose lines are that of the original plane + some rotation angle. Keras uses radians for the range. Check out [Wikipedia](https://en.wikipedia.org/wiki/Shear_mapping) for more information.

In [26]:
gen_sh = image.ImageDataGenerator(shear_range=0.1)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_sh, batch_size=img_batch_size)
create_and_run_simple_convnet()

Found 1803 images belonging to 10 classes.
Epoch 1/5
1803/1803 [==============================] - 32s - loss: 2.2506 - acc: 0.2768 - val_loss: 2.3205 - val_acc: 0.1138
Epoch 2/5
1803/1803 [==============================] - 24s - loss: 1.1949 - acc: 0.6534 - val_loss: 2.2926 - val_acc: 0.1652
Epoch 3/5
1803/1803 [==============================] - 26s - loss: 0.6900 - acc: 0.8630 - val_loss: 2.3246 - val_acc: 0.1674
Epoch 4/5
1803/1803 [==============================] - 25s - loss: 0.4346 - acc: 0.9401 - val_loss: 2.3643 - val_acc: 0.1317
Epoch 5/5
1803/1803 [==============================] - 26s - loss: 0.3091 - acc: 0.9651 - val_loss: 2.4020 - val_acc: 0.1138
Epoch 1/2
1803/1803 [==============================] - 31s - loss: 0.2057 - acc: 0.9900 - val_loss: 2.3838 - val_acc: 0.1317
Epoch 2/2
1803/1803 [==============================] - 25s - loss: 0.1608 - acc: 0.9967 - val_loss: 2.3734 - val_acc: 0.1473


#### Rotation Angle Augmentation
The image is simply randomly rotated by a degrees values in the range submitted.

In [27]:
gen_rt = image.ImageDataGenerator(rotation_range=15)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_rt, batch_size=img_batch_size)
create_and_run_simple_convnet()

Found 1803 images belonging to 10 classes.
Epoch 1/5
1803/1803 [==============================] - 31s - loss: 2.2863 - acc: 0.2795 - val_loss: 2.3322 - val_acc: 0.1384
Epoch 2/5
1803/1803 [==============================] - 24s - loss: 1.1066 - acc: 0.6883 - val_loss: 2.2767 - val_acc: 0.1429
Epoch 3/5
1803/1803 [==============================] - 26s - loss: 0.6399 - acc: 0.8741 - val_loss: 2.2968 - val_acc: 0.1384
Epoch 4/5
1803/1803 [==============================] - 25s - loss: 0.4168 - acc: 0.9401 - val_loss: 2.3031 - val_acc: 0.1518
Epoch 5/5
1803/1803 [==============================] - 26s - loss: 0.2768 - acc: 0.9834 - val_loss: 2.2669 - val_acc: 0.1875
Epoch 1/2
1803/1803 [==============================] - 32s - loss: 0.1918 - acc: 0.9917 - val_loss: 2.2409 - val_acc: 0.2098
Epoch 2/2
1803/1803 [==============================] - 25s - loss: 0.1495 - acc: 0.9956 - val_loss: 2.2105 - val_acc: 0.2232


#### Channel Shift Augmentation
Randomly shifting the R,G,B color channels 

In [28]:
gen_ch = image.ImageDataGenerator(channel_shift_range=10)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_ch, batch_size=img_batch_size)
create_and_run_simple_convnet()

Found 1803 images belonging to 10 classes.
Epoch 1/5
1803/1803 [==============================] - 33s - loss: 2.1001 - acc: 0.3178 - val_loss: 2.3843 - val_acc: 0.1116
Epoch 2/5
1803/1803 [==============================] - 24s - loss: 1.1510 - acc: 0.6705 - val_loss: 2.2682 - val_acc: 0.1674
Epoch 3/5
1803/1803 [==============================] - 23s - loss: 0.6533 - acc: 0.8686 - val_loss: 2.3285 - val_acc: 0.1473
Epoch 4/5
1803/1803 [==============================] - 24s - loss: 0.4210 - acc: 0.9445 - val_loss: 2.4200 - val_acc: 0.1362
Epoch 5/5
1803/1803 [==============================] - 25s - loss: 0.3038 - acc: 0.9673 - val_loss: 2.4801 - val_acc: 0.1518
Epoch 1/2
1803/1803 [==============================] - 32s - loss: 0.2168 - acc: 0.9872 - val_loss: 2.4694 - val_acc: 0.1562
Epoch 2/2
1803/1803 [==============================] - 24s - loss: 0.1654 - acc: 0.9945 - val_loss: 2.4729 - val_acc: 0.1830


#### Combining all augmentations

In [29]:
gen_all = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, shear_range=0.15, channel_shift_range=10, width_shift_range=0.1)
sample_train_batches = load_in_batches(sample_dir + '/train', shuffle=True, gen=gen_all, batch_size=img_batch_size)
model = create_and_run_simple_convnet()

Found 1803 images belonging to 10 classes.
Epoch 1/5
1803/1803 [==============================] - 32s - loss: 2.0674 - acc: 0.3200 - val_loss: 2.3414 - val_acc: 0.1652
Epoch 2/5
1803/1803 [==============================] - 24s - loss: 1.0091 - acc: 0.7232 - val_loss: 2.2138 - val_acc: 0.1942
Epoch 3/5
1803/1803 [==============================] - 26s - loss: 0.5925 - acc: 0.8913 - val_loss: 2.2152 - val_acc: 0.1763
Epoch 4/5
1803/1803 [==============================] - 28s - loss: 0.3844 - acc: 0.9468 - val_loss: 2.1976 - val_acc: 0.1942
Epoch 5/5
1803/1803 [==============================] - 29s - loss: 0.2558 - acc: 0.9756 - val_loss: 2.1718 - val_acc: 0.2232
Epoch 1/2
1803/1803 [==============================] - 32s - loss: 0.1768 - acc: 0.9906 - val_loss: 2.1450 - val_acc: 0.2612
Epoch 2/2
1803/1803 [==============================] - 27s - loss: 0.1384 - acc: 0.9983 - val_loss: 2.1134 - val_acc: 0.2812


Results are not looking that good. Let's change the learning rate and see if we improve the accuracy on validation sample set. In my original notebook the accuracy on training sample set was much lower. Maybe it's because the augmentations are applied at random and I was luckier in my previous spell as "better" augmentation were applied...

In [30]:
model.optimizer.lr = 0.0001
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=3, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/3
1803/1803 [==============================] - 32s - loss: 2.1090 - acc: 0.3361 - val_loss: 2.0389 - val_acc: 0.2946
Epoch 2/3
1803/1803 [==============================] - 29s - loss: 1.8628 - acc: 0.3988 - val_loss: 1.9312 - val_acc: 0.3549
Epoch 3/3
1803/1803 [==============================] - 27s - loss: 1.6985 - acc: 0.4365 - val_loss: 1.8381 - val_acc: 0.4018


This is encouraging... Let's try more epochs...

In [31]:
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=15, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/15
1803/1803 [==============================] - 34s - loss: 1.5848 - acc: 0.4759 - val_loss: 1.7262 - val_acc: 0.4598
Epoch 2/15
1803/1803 [==============================] - 28s - loss: 1.4848 - acc: 0.5058 - val_loss: 1.7219 - val_acc: 0.4330
Epoch 3/15
1803/1803 [==============================] - 27s - loss: 1.4427 - acc: 0.5197 - val_loss: 1.6885 - val_acc: 0.4643
Epoch 4/15
1803/1803 [==============================] - 26s - loss: 1.3390 - acc: 0.5591 - val_loss: 1.6704 - val_acc: 0.4821
Epoch 5/15
1803/1803 [==============================] - 28s - loss: 1.2676 - acc: 0.5801 - val_loss: 1.6385 - val_acc: 0.5022
Epoch 6/15
1803/1803 [==============================] - 27s - loss: 1.2019 - acc: 0.6140 - val_loss: 1.6648 - val_acc: 0.5268
Epoch 7/15
1803/1803 [==============================] - 26s - loss: 1.2263 - acc: 0.6001 - val_loss: 1.6682 - val_acc: 0.5513
Epoch 8/15
1803/1803 [==============================] - 28s - loss: 1.1575 - acc: 0.6251 - val_loss: 1.6483 - val_acc:

In [32]:
# Increasing learning rate... Let's see what happens
model.optimizer.lr = 0.001
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=5, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/5
1803/1803 [==============================] - 31s - loss: 0.9475 - acc: 0.7005 - val_loss: 1.5854 - val_acc: 0.5379
Epoch 2/5
1803/1803 [==============================] - 27s - loss: 0.9099 - acc: 0.6999 - val_loss: 1.5498 - val_acc: 0.5826
Epoch 3/5
1803/1803 [==============================] - 29s - loss: 0.9000 - acc: 0.7232 - val_loss: 1.5532 - val_acc: 0.5714
Epoch 4/5
1803/1803 [==============================] - 28s - loss: 0.8898 - acc: 0.7138 - val_loss: 1.5516 - val_acc: 0.5915
Epoch 5/5
1803/1803 [==============================] - 27s - loss: 0.8577 - acc: 0.7343 - val_loss: 1.6051 - val_acc: 0.5915


Our model seems to be reaching its limits - we could probably get slightly better results but we are clearly seeing diminishing returns on the validation set. Let's try another 5 epochs and stop.

In [35]:
model.optimizer.lr = 0.00001
model.fit_generator(sample_train_batches, samples_per_epoch=sample_train_batches.nb_sample, nb_epoch=5, 
                   validation_data=sample_val_batches, nb_val_samples=sample_val_batches.nb_sample, verbose=1)

Epoch 1/5
1803/1803 [==============================] - 31s - loss: 0.7018 - acc: 0.7937 - val_loss: 1.5879 - val_acc: 0.5647
Epoch 2/5
1803/1803 [==============================] - 27s - loss: 0.6374 - acc: 0.8125 - val_loss: 1.6037 - val_acc: 0.5960
Epoch 3/5
1803/1803 [==============================] - 27s - loss: 0.6662 - acc: 0.8081 - val_loss: 1.5528 - val_acc: 0.6027
Epoch 4/5
1803/1803 [==============================] - 29s - loss: 0.6656 - acc: 0.7948 - val_loss: 1.5072 - val_acc: 0.5781
Epoch 5/5
1803/1803 [==============================] - 27s - loss: 0.6499 - acc: 0.8197 - val_loss: 1.4717 - val_acc: 0.5915


The results are still very encouraging for such a simple model! Yet it's definitely reached its limits. We must now use the full dataset to try more powerful techniques and make use of a pre-trained VGG network